<a href="https://colab.research.google.com/github/caavila96/gesproy23/blob/main/Analisis_Gesproy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Análisis de la base de datos de Gestión de Proyectos del Sistema General de Regalias - Gesproy**

**Autor:** Camilo Andrés Avila Carreño

El GESPROY es un aplicativo o plataforma tecnológica dispuesta para el reporte y seguimiento de información de los proyectos ejecutados con recursos del SGR. El ente ejecutor se encarga de reportar la información correspondiente a programación, contratación y ejecución de las actividades de los proyectos.

Esta base de datos contiene la totalidad de proyectos aprobados desde el 1 de enero de 2012 hasta la fecha del último corte, con la información actualizada a este último corte, así como, Contiene el desglose fuentes de financiación actual de los proyectos incluyendo los resultantes del proceso de homologación de fuentes

### **Librerías**

In [123]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from google.colab import data_table
from datetime import date, datetime
from dateutil.relativedelta import relativedelta # Sumar fechas 
import unicodedata #Eliminar los acentos (normalizar)


### **Importación de información**

In [2]:
url = "https://raw.githubusercontent.com/caavila96/gesproy23/main/BASE/Base_Proyectos_15_03_2023.xlsx"
gesp = pd.read_excel(url)
gesp.head()

,BPIN,INSTANCIA DE APROBACIÓN INICIAL,TIPO DE INSTANCIA INICIAL,REGION ENTIDAD QUE MÁS APORTA RECURSOS,DEPARTAMENTO ENTIDAD QUE MÁS APORTA RECURSOS,CODIGO ENTIDAD TERRITORIAL QUE APORTA MÁS RECURSOS,ENTIDAD TERRITORIAL QUE APORTA MÁS RECURSOS,REGION EJECUTOR,DEPARTAMENTO EJECUTOR,CÓDIGO EJECUTOR,...,INICIO DE OBLIGACIÓN COMO EJECUTOR,# DE DENUNCIAS CIUDADANAS RECIBIDAS,CON APORTE CIUDADANO,DECISIÓN MOTIVADA DE NO CONTINUIDAD,CLASIFICACIÓN PLAZO PARA EJECUCIÓN,Rango valor SGR,Rango total proyecto,Rango avance fisico,Rango avance financiero,Rango total pagos
0,2022006910134,DEPARTAMENTO DE AMAZONAS,DEPARTAMENTAL,CENTRO SUR,AMAZONAS,91000,AMAZONAS,CENTRO SUR,AMAZONAS,6666213,...,2023-03-06,0,NaN,N,NaN,Entre $1.000 y $5.000 millones,Entre $1.000 y $5.000 millones,Sin avance físico,Sin avance financiero,Menor a $500 millones
1,2012000060032,REGIÓN CENTRO SUR,REGIONAL,CENTRO SUR,AMAZONAS,91000,AMAZONAS,CENTRO SUR,AMAZONAS,91000,...,2013-02-27,0,NaN,N,NaN,Menor a $500 millones,Menor a $500 millones,entre 80% y 100%,entre 80% y 100%,Menor a $500 millones
2,2012000060033,REGIÓN CENTRO SUR,REGIONAL,CENTRO SUR,AMAZONAS,91000,AMAZONAS,CENTRO SUR,AMAZONAS,91000,...,2013-02-27,0,NaN,N,NaN,Entre $500 y $1.000 millones,Entre $500 y $1.000 millones,entre 80% y 100%,Entre 50% y 80%,Menor a $500 millones
3,2012000100057,"DEPARTAMENTO ADMINISTRATIVO DE LA CIENCIA, LA ...",CIENCIA Y TECNOLOGÍA,CENTRO SUR,AMAZONAS,91000,AMAZONAS,CENTRO SUR,AMAZONAS,91000,...,2012-12-20,0,NaN,N,Plazo Vencido,Menor a $500 millones,Menor a $500 millones,Entre 50% y 80%,entre 80% y 100%,Menor a $500 millones
4,2012000100158,"DEPARTAMENTO ADMINISTRATIVO DE LA CIENCIA, LA ...",CIENCIA Y TECNOLOGÍA,CENTRO SUR,AMAZONAS,91000,AMAZONAS,CENTRO SUR,AMAZONAS,91000,...,2012-12-20,0,NaN,N,NaN,Entre $500 y $1.000 millones,Entre $1.000 y $5.000 millones,entre 80% y 100%,entre 80% y 100%,Entre $500 y $1.000 millones


## 1. Reconocimiento del DataFrame. Tamaño, naturaleza de los datos, nombre de columnas, formato de los datos, etc.

### 1.1 Tamaño del DataFrame

In [3]:
shp = gesp.shape
sz = gesp.size

print(f'La base de datos contien un total de {shp[0]:,} filas y {shp[1]} columnas, para un total de {sz:,} datos almacenados en la base de datos' )

La base de datos contien un total de 26,213 filas y 157 columnas, para un total de 4,115,441 datos almacenados en la base de datos


### 1.2 Naturaleza de los datos 

In [ ]:
gesp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26213 entries, 0 to 26212
Columns: 157 entries, BPIN to Rango total pagos
dtypes: datetime64[ns](20), float64(57), int64(24), object(56)
memory usage: 31.4+ MB


### 1.3 Nombre de las columnas y formato de los datos

In [11]:
table = pd.DataFrame({'Nombre_columna': gesp.columns,
                      'Formato_columna': gesp.dtypes}).reset_index(drop = True)

data_table.enable_dataframe_formatter()
table.head()


,Nombre_columna,Formato_columna
0,BPIN,object
1,INSTANCIA DE APROBACIÓN INICIAL,object
2,TIPO DE INSTANCIA INICIAL,object
3,REGION ENTIDAD QUE MÁS APORTA RECURSOS,object
4,DEPARTAMENTO ENTIDAD QUE MÁS APORTA RECURSOS,object


Teniendo en cuenta que los nombres de las variables presentan caracteres especiales, no hay homogenidad entre mayusculas y minúsculas y tienen nombres muy extensos es necesario realizar ajustes a estos nombres,  

In [25]:
# Se genera una función para normalizar los nombres, quitar tildes, otros caracteres especiales y hacer un diccionario con el nuevo nombre de las variables
def quitar_accentos(s):
    #Elimina las tíldes
    nuevo_nom = ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
    # Todos los caracteres en mayúscula
    nuevo_nom = nuevo_nom.lower()
    # Se reemlazan los caracteres especiales excepto los " " y "_"
    nuevo_nom = re.sub('[^A-Za-z0-9 _]+', '', nuevo_nom)
    # Se reemplaza los " " por "_"
    nuevo_nom = re.sub(' ', '_', nuevo_nom)
    return nuevo_nom


In [26]:
# Se crea una nueva variable con los nuevos nombres   
table["Nombre_columna_dic"] = table["Nombre_columna"].apply(quitar_accentos) #apply aplica la función en toda la serie
table

,Nombre_columna,Formato_columna,Nombre_columna_dic
0,BPIN,object,bpin
1,INSTANCIA DE APROBACIÓN INICIAL,object,instancia_de_aprobacion_inicial
2,TIPO DE INSTANCIA INICIAL,object,tipo_de_instancia_inicial
3,REGION ENTIDAD QUE MÁS APORTA RECURSOS,object,region_entidad_que_mas_aporta_recursos
4,DEPARTAMENTO ENTIDAD QUE MÁS APORTA RECURSOS,object,departamento_entidad_que_mas_aporta_recursos
...,...,...,...
152,Rango valor SGR,object,rango_valor_sgr
153,Rango total proyecto,object,rango_total_proyecto
154,Rango avance fisico,object,rango_avance_fisico
155,Rango avance financiero,object,rango_avance_financiero


In [51]:
# Para mayor trazabilidad se genera un excel como diccionario de datos en el repositorio de Github
table.to_csv("diccionario_v2.csv",encoding="ISO-8859-1")

In [43]:
# Se genera el cambio de nombre en todas la variables 
gesp = gesp.rename(
    columns = dict(zip(table.Nombre_columna,table.Nombre_columna_dic)) #zip is an iterator of tuples where the first item in each passed iterator is paired together, and then the second item in each passed iterator are paired together etc
) 

In [45]:
gesp.head()

,bpin,instancia_de_aprobacion_inicial,tipo_de_instancia_inicial,region_entidad_que_mas_aporta_recursos,departamento_entidad_que_mas_aporta_recursos,codigo_entidad_territorial_que_aporta_mas_recursos,entidad_territorial_que_aporta_mas_recursos,region_ejecutor,departamento_ejecutor,codigo_ejecutor,...,inicio_de_obligacion_como_ejecutor,_de_denuncias_ciudadanas_recibidas,con_aporte_ciudadano,decision_motivada_de_no_continuidad,clasificacion_plazo_para_ejecucion,rango_valor_sgr,rango_total_proyecto,rango_avance_fisico,rango_avance_financiero,rango_total_pagos
0,2022006910134,DEPARTAMENTO DE AMAZONAS,DEPARTAMENTAL,CENTRO SUR,AMAZONAS,91000,AMAZONAS,CENTRO SUR,AMAZONAS,6666213,...,2023-03-06,0,NaN,N,NaN,Entre $1.000 y $5.000 millones,Entre $1.000 y $5.000 millones,Sin avance físico,Sin avance financiero,Menor a $500 millones
1,2012000060032,REGIÓN CENTRO SUR,REGIONAL,CENTRO SUR,AMAZONAS,91000,AMAZONAS,CENTRO SUR,AMAZONAS,91000,...,2013-02-27,0,NaN,N,NaN,Menor a $500 millones,Menor a $500 millones,entre 80% y 100%,entre 80% y 100%,Menor a $500 millones
2,2012000060033,REGIÓN CENTRO SUR,REGIONAL,CENTRO SUR,AMAZONAS,91000,AMAZONAS,CENTRO SUR,AMAZONAS,91000,...,2013-02-27,0,NaN,N,NaN,Entre $500 y $1.000 millones,Entre $500 y $1.000 millones,entre 80% y 100%,Entre 50% y 80%,Menor a $500 millones
3,2012000100057,"DEPARTAMENTO ADMINISTRATIVO DE LA CIENCIA, LA ...",CIENCIA Y TECNOLOGÍA,CENTRO SUR,AMAZONAS,91000,AMAZONAS,CENTRO SUR,AMAZONAS,91000,...,2012-12-20,0,NaN,N,Plazo Vencido,Menor a $500 millones,Menor a $500 millones,Entre 50% y 80%,entre 80% y 100%,Menor a $500 millones
4,2012000100158,"DEPARTAMENTO ADMINISTRATIVO DE LA CIENCIA, LA ...",CIENCIA Y TECNOLOGÍA,CENTRO SUR,AMAZONAS,91000,AMAZONAS,CENTRO SUR,AMAZONAS,91000,...,2012-12-20,0,NaN,N,NaN,Entre $500 y $1.000 millones,Entre $1.000 y $5.000 millones,entre 80% y 100%,entre 80% y 100%,Entre $500 y $1.000 millones


## 2. Asegurar que las variables numéricas estén en el formato correcto.

In [52]:
gesp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26213 entries, 0 to 26212
Columns: 157 entries, bpin to rango_total_pagos
dtypes: datetime64[ns](20), float64(57), int64(24), object(56)
memory usage: 32.6+ MB


El proceso de revisión inicialmente se realiza para las 56 columnas importadas como **"objects"** este tipo de formato sirve para texto o un mixto entre variables numéricas y no numericas. 

Este tipo de variable puede reducir la capacidad de generar algunas funciones propias de los string o de las variables numéricas, por ende es mejor realizar una reclasificación de la misma. 

In [61]:
# se imprimen las variables con tipo objects para una identificación inicial de las mismas
pd.set_option('display.max_columns', None) #Elimina la cantidad de columnas que se pueden mostrar

gesp.select_dtypes(include='object').head(5)




,bpin,instancia_de_aprobacion_inicial,tipo_de_instancia_inicial,region_entidad_que_mas_aporta_recursos,departamento_entidad_que_mas_aporta_recursos,entidad_territorial_que_aporta_mas_recursos,region_ejecutor,departamento_ejecutor,entidad_ejecutora,tipo_ejecutor,tipo_ejecutor_detallado,sector_suifp,programa,subprograma,nombre_del_proyecto,estado_detalle,estado_general,tiempo_desde_la_aprobacion,en_riesgo_de_liberacion_de_recursos,implementacion_acuerdo_paz,proyectos_reactivacion_economica,marca_covid,minima_fecha_proceso_precontractual,fase,destino_recursos,enfoque_diferencial,entidad_designada_contratar_interventoria,critico_persistente,abandonado_suspendido_indefinidamente_o_de_forma_reiterada_yo_con_retrasos_injustificados,insuficiencias_tecnicas_graves,falta_de_sostenibilidad_funcionalidad_yo_operatividad,desclasificado_como_critico,requiere_plan_de_contingencia,plan_de_contingencia_cumplido,plan_de_contigencia_no_cumplido,plan_de_contingencia_en_seguimiento,drones,ultimo_ejercicio_de_control_social,oficina_regional,operaciones_credito_publico,proyectos_obras_de_inf_fisica_en_ipes,planes_de_desarrollo_con_enfoque_territorial__pdet_segun_entidad_ejecutora,subregiones_pdet_proyectos_aprobados_ocad_paz_segun_localizacion,consolidado_pdet,proyecto_entidad_priorizada_fortalecimiento,marca_municipios_mas_pobres,marca_entidad_privada,alcance,proyecto_cesg,decision_motivada_de_no_continuidad,clasificacion_plazo_para_ejecucion,rango_valor_sgr,rango_total_proyecto,rango_avance_fisico,rango_avance_financiero,rango_total_pagos
0,2022006910134,DEPARTAMENTO DE AMAZONAS,DEPARTAMENTAL,CENTRO SUR,AMAZONAS,AMAZONAS,CENTRO SUR,AMAZONAS,AMAZONAS DESARROLLO INTELIGENTE - ADI SAS ESP,OTROS,E.S.P,EDUCACIÓN,"CALIDAD, COBERTURA Y FORTALECIMIENTO DE LA EDU...",INTERSUBSECTORIAL EDUCACION,CONSTRUCCIÓN DE AULAS EN LA I.E. FRANCISCO ROS...,SIN CONTRATAR,SIN CONTRATAR,1.De 1 a 6 meses,NaN,NO,NO,N,NaN,FACTIBILIDAD - FASE 3,EJECUCION,Sin Enfoque Diferencial,NaN,NaN,NaN,NaN,NaN,NO,NaN,NaN,NaN,NaN,NO,NaN,CENTRO SUR,NO,NaN,NaN,NaN,NaN,NO,NO,NO,NaN,NaN,N,NaN,Entre $1.000 y $5.000 millones,Entre $1.000 y $5.000 millones,Sin avance físico,Sin avance financiero,Menor a $500 millones
1,2012000060032,REGIÓN CENTRO SUR,REGIONAL,CENTRO SUR,AMAZONAS,AMAZONAS,CENTRO SUR,AMAZONAS,DEPARTAMENTO DEL AMAZONAS,DEPARTAMENTO,Departamento,EDUCACIÓN,ESTUDIOS DE PREINVERSION,"EDUCACION PREESCOLAR, BASICA Y MEDIA",ESTUDIOS Y DISEÑOS PARA LA MODERNIZACION DE CI...,TERMINADO,TERMINADO,4.Más de 12 meses,NaN,NO,NO,N,2013-12-06 00:00:00,PREFACTIBILIDAD - FASE 2,ESTUDIOS,Sin Enfoque Diferencial,NaN,NaN,NaN,NaN,NaN,NO,No,NaN,NaN,NaN,NO,NaN,CENTRO SUR,NO,NaN,NaN,NaN,NaN,SI,NO,NO,REALIZAR LOS ESTUDIOS Y DISEÑOS PARA LA MODERN...,NaN,N,NaN,Menor a $500 millones,Menor a $500 millones,entre 80% y 100%,entre 80% y 100%,Menor a $500 millones
2,2012000060033,REGIÓN CENTRO SUR,REGIONAL,CENTRO SUR,AMAZONAS,AMAZONAS,CENTRO SUR,AMAZONAS,DEPARTAMENTO DEL AMAZONAS,DEPARTAMENTO,Departamento,SALUD Y PROTECCIÓN SOCIAL,ESTUDIOS DE PREINVERSION,PRESTACIÓN DE SERVICIOS EN SALUD,ESTUDIOS Y DISEÑOS PARA LA MODERNIZACION DE LO...,TERMINADO,TERMINADO,4.Más de 12 meses,NaN,NO,NO,N,2013-12-06 00:00:00,PREFACTIBILIDAD - FASE 2,ESTUDIOS,Sin Enfoque Diferencial,NaN,NaN,NaN,NaN,NaN,NO,No,NaN,NaN,NaN,NO,NaN,CENTRO SUR,NO,NaN,NaN,NaN,NaN,SI,NO,NO,MODERNIZACIÓN A LA RED PRESTADORA DE SERVICIOS...,NaN,N,NaN,Entre $500 y $1.000 millones,Entre $500 y $1.000 millones,entre 80% y 100%,Entre 50% y 80%,Menor a $500 millones
3,2012000100057,"DEPARTAMENTO ADMINISTRATIVO DE LA CIENCIA, LA ...",CIENCIA Y TECNOLOGÍA,CENTRO SUR,AMAZONAS,AMAZONAS,CENTRO SUR,AMAZONAS,DEPARTAMENTO DEL AMAZONAS,DEPARTAMENTO,Departamento,"CIENCIA, TECNOLOGÍA E INNOVACIÓN","INVESTIGACION BASICA, APLICADA Y ESTUDIOS",BIODIVERSIDAD Y SUS SERVICIOS ECOSISTEMICOS,INVESTIGACIÓN EVALUACION DEL CAMBIO GLOBAL EN ...,CONTRATADO EN EJECUCIÓN,EN EJECUCIÓN,4.Más de 12 meses,NaN,NO,NO,N,2013-11-08 00:00:00,PERFIL - FASE 1,ESTUDIOS,Sin Enfoque Diferencial,NaN,NaN,NaN,

En la inspección visual inicialmente solo la variable minima_fecha_proceso_precontractual sería una fecha, ahora bien pandas tiene la función convert_dtypes que evalua las variables y las pasa a un al mejor dtype teniendo en cuenta su naturaleza

In [101]:
#se aplica la función convert_dtypes creando un segundo dataframe y se comparan los dtypes
gesp_2 = gesp.convert_dtypes()


print(f"Los dtypes ANTES de aplicar el convert_dtypes es \n {gesp.dtypes.value_counts()} \n \n" ) 
print(f"Los dtypes DESPUÉS de aplicar el convert_dtypes es \n {gesp_2.dtypes.value_counts()}" ) 


Los dtypes ANTES de aplicar el convert_dtypes es 
 float64           57
object            56
int64             24
datetime64[ns]    20
dtype: int64 
 

Los dtypes DESPUÉS de aplicar el convert_dtypes es 
 string            53
Float64           47
Int64             34
datetime64[ns]    20
object             3
dtype: int64


Se revisan cada uno de los tipos para ver una efectiva calificación.

*  Objects



In [107]:
#Se revisan los tipo object restantes
gesp_2.select_dtypes(include='object').head(5)

,minima_fecha_proceso_precontractual,alcance
0,NaN,NaN
1,2013-12-06 00:00:00,REALIZAR LOS ESTUDIOS Y DISEÑOS PARA LA MODERN...
2,2013-12-06 00:00:00,MODERNIZACIÓN A LA RED PRESTADORA DE SERVICIOS...
3,2013-11-08 00:00:00,ANALIZAR Y CUANTIFICAR EL CAMBIO SOCIO-AMBIENT...
4,2013-11-08 00:00:00,CUANTIFICAR LA OFERTA NATURAL DE ESPECIES POR ...


In [106]:
# La varible BPIN a pesar de ser numérica al ser una llave para la identificacióm de los proyectos se convierte a string.
gesp_2["bpin"] = gesp_2["bpin"].astype("string")


In [118]:
#La variable minima_fecha_proceso_precontractual es tipo fecha, pero al intentar realizar el cambio de tipo de fecha genera un error respecto a un dato, para ello se revisa cual puede ser el problema
gesp_2.minima_fecha_proceso_precontractual.groupby(gesp_2.minima_fecha_proceso_precontractual).count() 


minima_fecha_proceso_precontractual
1999-01-01 00:00:00    1
2006-05-26 00:00:00    2
2006-06-06 00:00:00    1
2008-08-28 00:00:00    1
2010-03-09 00:00:00    1
                      ..
2023-03-10 00:00:00    2
2023-03-13 00:00:00    1
2023-03-14 00:00:00    1
2023-03-15 00:00:00    2
0/01/1900              1
Name: minima_fecha_proceso_precontractual, Length: 3009, dtype: int64

In [129]:
# se observa el valor 0/01/1900 que no puede convertirse en fecha, claramente es un valor ayípico, para solucionar el problema y según establece el diccionario de la base de datos 
# la fecha mínima para inicial el proceso precontractual son los 6 primeros meses después de la aprobación se identifica la observacion que tiene el problema y se reemplaza el dato con los 6 meses después
# de el inicio de la aprobación

gesp_2[gesp_2.minima_fecha_proceso_precontractual == "0/01/1900"]["minima_fecha_proceso_precontractual"] = gesp_2[gesp_2.minima_fecha_proceso_precontractual == "0/01/1900"].fecha_aprobacion +  pd.DateOffset(months = 6)

<ipython-input-129-6a5d832c374b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gesp_2[gesp_2.minima_fecha_proceso_precontractual == "0/01/1900"]["minima_fecha_proceso_precontractual"] = gesp_2[gesp_2.minima_fecha_proceso_precontractual == "0/01/1900"].fecha_aprobacion +  pd.DateOffset(months = 6)


In [130]:
gesp_2.minima_fecha_proceso_precontractual.groupby(gesp_2.minima_fecha_proceso_precontractual).count() 

minima_fecha_proceso_precontractual
1999-01-01 00:00:00    1
2006-05-26 00:00:00    2
2006-06-06 00:00:00    1
2008-08-28 00:00:00    1
2010-03-09 00:00:00    1
                      ..
2023-03-10 00:00:00    2
2023-03-13 00:00:00    1
2023-03-14 00:00:00    1
2023-03-15 00:00:00    2
0/01/1900              1
Name: minima_fecha_proceso_precontractual, Length: 3009, dtype: int64

## 3. Reconocimiento y limpieza de los datos faltantes. Estimación del porcentaje de datos eliminados.

In [126]:

gesp_2[gesp_2.minima_fecha_proceso_precontractual == "0/01/1900"].fecha_aprobacion +  pd.DateOffset(months = 6)

8596   2015-12-02
Name: fecha_aprobacion, dtype: datetime64[ns]

8596    0/01/1900
Name: minima_fecha_proceso_precontractual, dtype: object

In [110]:
gesp_2["minima_fecha_proceso_precontractual"][8596]

'0/01/1900'

## 4. Calcular alguna cantidad de interés en una nueva columna.

TypeError: ignored

## 5. Genere nuevos DataFrames agrupando datos según características de interés, tango usando .groupby() como filtros. Una dos de los DataFrames generados por filtro y agrupamiento, inteprete cuidadosamente los datos obtenidos.

## 6. Graficar variables numéricas matizando según variables categóricas (hue).

## 7. Construir tablas pivote y hacer gráficas de calor de las mismas.

## 8. Hacer al menos dos gráficas con datos categóricos.

## 9. Graficar y analizar la correlación de las variables numéricas del DataSet.

## 10. Use *expresiones regulares* ( $\hat{ }$ , .* , \$ , etc. ) para filtrar dependiendo contenidos de texto y grafique los resultados obtenidos. 